In [1]:
import gspread
import pandas as pd

In [2]:
from oauth2client.service_account import ServiceAccountCredentials

In [67]:
# This cell of code sets up the authorization of the API
scope = ['https://spreadsheets.google.com/feeds']
creds = ServiceAccountCredentials.from_json_keyfile_name('C:\\Users\\riche\\Documents\\DataScience-Python3\\client_secret.json',scope)
client= gspread.authorize(creds)
client

In [68]:
# Using the spreadsheet key to open the workbook and the worksheet id to open the specific sheet
sheet = client.open_by_key('1akG4vWlTykYY-ewhLdwOXTHk629ZPqB_w0ZhrvRHyy8').get_worksheet(0)
# Getting all rows into a list 
result = sheet.get_all_values()

In [81]:
# The sheet has some rows on top that are not in format - these are being removed and a new list being made
result_parsed = result[4:]

In [70]:
# Importing the list into a dataframe with the first element of the list as the column names
df= pd.DataFrame(result_parsed,columns=result_parsed[0])
df.drop(labels=[0,1], axis=0,inplace=True)
df.reset_index(inplace=True)
df['index']=df['index'].apply(lambda x:x-2)

In [71]:
df.head()

,index,Day,Date,Time,Activity,Sleep Duration,DateTime,,
0,0,365,3/5/2018,9:55 AM,Nap Start,,3/5/2018 9:55:00,,
1,1,365,3/5/2018,11:05 AM,Nap End,1:10:00,3/5/2018 11:05:00,,
2,2,365,3/5/2018,3:11 PM,Nap Start,,3/5/2018 15:11:00,,
3,3,365,3/5/2018,4:27 PM,Nap End,1:16:00,3/5/2018 16:27:00,,
4,4,365,3/5/2018,8:00 PM,Nap Start,,3/5/2018 20:00:00,,


In [38]:
import datetime as dt

In [72]:
#Sample use of function strptime to get a datetime from a string
date = dt.datetime.strptime(df['DateTime'][1834],'%m/%d/%Y %H:%M:%S')
date

datetime.datetime(2019, 3, 14, 19, 0)

In [73]:
df.head()

,index,Day,Date,Time,Activity,Sleep Duration,DateTime,,
0,0,365,3/5/2018,9:55 AM,Nap Start,,3/5/2018 9:55:00,,
1,1,365,3/5/2018,11:05 AM,Nap End,1:10:00,3/5/2018 11:05:00,,
2,2,365,3/5/2018,3:11 PM,Nap Start,,3/5/2018 15:11:00,,
3,3,365,3/5/2018,4:27 PM,Nap End,1:16:00,3/5/2018 16:27:00,,
4,4,365,3/5/2018,8:00 PM,Nap Start,,3/5/2018 20:00:00,,


In [74]:
difflist = []
for index in df.index:
#     print("Current index:", index,df['Activity'][index],df['DateTime'][index])
    
    if index == 0:
        difflist.append(dt.datetime.strptime(df['DateTime'][index],'%m/%d/%Y %H:%M:%S')-dt.datetime.strptime(df['DateTime'][index],'%m/%d/%Y %H:%M:%S'))
    else:
        difflist.append(dt.datetime.strptime(df['DateTime'][index],'%m/%d/%Y %H:%M:%S')-dt.datetime.strptime(df['DateTime'][index-1],'%m/%d/%Y %H:%M:%S'))
    

In [75]:
df_diff = pd.DataFrame(difflist,columns=['Sleep Time'])
df_diff.head()

,Sleep Time
0,00:00:00
1,01:10:00
2,04:06:00
3,01:16:00
4,03:33:00


In [76]:
cdf = pd.concat([df,df_diff], axis=1, join='outer')
cdf.tail()

,index,Day,Date,Time,Activity,Sleep Duration,DateTime,,,Sleep Time
1845,1845,742,3/17/2019,1:21 PM,Nap End,1:13:00,3/17/2019 13:21:00,,,01:13:00
1846,1846,742,3/17/2019,7:11 PM,Nap Start,,3/17/2019 19:11:00,,,05:50:00
1847,1847,743,3/18/2019,5:56 AM,Nap End,10:45:00,3/18/2019 5:56:00,,,10:45:00
1848,1848,743,3/18/2019,12:14 PM,Nap Start,,3/18/2019 12:14:00,,,06:18:00
1849,1849,743,3/18/2019,2:40 PM,Nap End,2:26:00,3/18/2019 14:40:00,,,02:26:00


In [47]:
# OPtional code to add a column with DateTime in datatime datatype
# df['DateStrtoDatetime'] = df['DateTime'].apply(lambda x:dt.datetime.strptime(x,'%m/%d/%Y %H:%M:%S')) 

In [77]:
end_df = cdf[cdf['Activity']=="Nap End"]
end_df.head()

,index,Day,Date,Time,Activity,Sleep Duration,DateTime,,,Sleep Time
1,1,365,3/5/2018,11:05 AM,Nap End,1:10:00,3/5/2018 11:05:00,,,01:10:00
3,3,365,3/5/2018,4:27 PM,Nap End,1:16:00,3/5/2018 16:27:00,,,01:16:00
5,5,366,3/6/2018,5:57 AM,Nap End,9:57:00,3/6/2018 5:57:00,,,09:57:00
7,7,366,3/6/2018,10:50 AM,Nap End,1:45:00,3/6/2018 10:50:00,,,01:45:00
9,9,366,3/6/2018,4:05 PM,Nap End,1:35:00,3/6/2018 16:05:00,,,01:35:00


In [78]:
end_df['Sleep Time'].min()

Timedelta('0 days 00:10:00')

In [92]:
cdf[(cdf['Sleep Time']>'0 days 13:00:00')]
        

,index,Day,Date,Time,Activity,Sleep Duration,DateTime,,,Sleep Time
1459,1459,644,12/9/2018,8:07 AM,Nap End,13:39:00,12/9/2018 8:07:00,,,13:39:00
1796,1796,730,3/5/2019,7:08 PM,Nap Start,,3/5/2019 19:08:00,,,14:00:00


In [78]:
df.index

RangeIndex(start=0, stop=1838, step=1)